In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2

import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.initializers import glorot_uniform
from tensorflow.keras.utils import plot_model
from IPython.display import display
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint, LearningRateScheduler

from keras.optimizers import SGD, Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping

In [2]:
def display_image(image_array, image_number):
    acrosome_labels = np.load("mhsma\\y_acrosome_train.npy")
    head_labels = np.load("mhsma\\y_head_train.npy")
    tail_labels = np.load("mhsma\\y_tail_train.npy")
    vacuole_labels = np.load("mhsma\\y_vacuole_train.npy")

    acrosome_label = acrosome_labels[image_number]
    head_label = head_labels[image_number]
    tail_label = tail_labels[image_number]
    vacuole_label = vacuole_labels[image_number]
    print(f"Acrosome: {acrosome_label}, Head: {head_label}, Tail: {tail_label}, Vacuole: {vacuole_label}")
    plt.imshow(image_array, cmap='gray')
    plt.axis('off') 
    plt.title(f"Image {image_number}")
    plt.show()

In [3]:
x_train=np.load('mhsma\\x_128_train.npy')
x_test=np.load('mhsma\\x_128_test.npy')
x_valid=np.load('mhsma\\x_128_valid.npy')

In [4]:
# x_train = tf.cast(x_train, tf.float32)
# x_test = tf.cast(x_test, tf.float32)
# x_valid = tf.cast(x_valid, tf.float32)

In [5]:
y_train=np.load('mhsma\\y_head_train.npy')
y_test=np.load('mhsma\\y_head_test.npy')
y_valid=np.load('mhsma\\y_head_valid.npy')

In [6]:
# y_train = tf.cast(y_train, tf.float32)
# y_test = tf.cast(y_test, tf.float32)
# y_valid = tf.cast(y_valid, tf.float32)

In [ ]:
datagen = ImageDataGenerator(featurewise_center=True,
    samplewise_center=False,
    featurewise_std_normalization=True,
    samplewise_std_normalization=False,
    zca_whitening=False,
    rotation_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    fill_mode='nearest',
    cval=0.2,
    horizontal_flip=True,
    vertical_flip=False,
    rescale=1./255,
    preprocessing_function=None)

In [ ]:
x_train_reshaped = x_train.reshape(1000, 128, 128, 1)
x_valid_reshaped = x_valid.reshape(240, 128, 128, 1)

In [ ]:
x_train_flow = datagen.flow(x_train_reshaped, y_train,batch_size=30)
x_valid_flow = datagen.flow(x_valid_reshaped, y_valid,batch_size=30)

In [9]:
from transformers import AutoImageProcessor
image_processor  = AutoImageProcessor.from_pretrained(model_checkpoint)

c:\Users\Jayesh\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Jayesh\AppData\Local\Programs\Python\Python39\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Jayesh\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see a

In [10]:
from transformers import SwinConfig, SwinModel
configuration = SwinConfig()
model = SwinModel(configuration)
configuration = model.config

c:\Users\Jayesh\AppData\Local\Programs\Python\Python39\lib\site-packages\bitsandbytes\cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


'NoneType' object has no attribute 'cadam32bit_grad_fp32'


In [11]:
import torch
image_processor = AutoImageProcessor.from_pretrained("microsoft/swin-tiny-patch4-window7-224")
model = SwinModel.from_pretrained("microsoft/swin-tiny-patch4-window7-224")

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


In [8]:
basemodel=InceptionResNetV2(weights='imagenet',include_top=False, input_tensor=Input(shape=(128, 128, 1)))

ValueError: Cannot assign value to variable ' conv2d/kernel:0': Shape mismatch.The variable shape (3, 3, 1, 32), and the assigned value shape (32, 3, 3, 3) are incompatible.

In [ ]:
basemodel.trainable=False

In [ ]:
x = basemodel.output
x = GlobalAveragePooling2D()(x)
x=Flatten()(x)
x=Dense(256,activation='relu')(x)
x = Dropout(0.3)(x)
x=Dense(128,activation='relu')(x)
x = Dropout(0.3)(x)
predictions = Dense(1, activation= 'sigmoid')(x)
model = Model(inputs = basemodel.input, outputs = predictions)

In [ ]:

adam = Adam(lr=0.0001)
model.compile(optimizer= SGD(lr=0.01), loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:

model.input.shape


In [ ]:
checkpoint = ModelCheckpoint("rvgg16_1.h5", monitor='val_accuracy', verbose=1, save_best_only=True, save_weights_only=False, mode='auto')
early = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=34, verbose=1, mode='auto')
hist = model.fit(steps_per_epoch=34,x=x_train_flow, validation_data= x_valid_flow,epochs=25,callbacks=[checkpoint,early])

In [ ]:
npy_file_path = 'mhsma\\x_128_train.npy'
images = np.load(npy_file_path)
image_number = 31
display_image(images[image_number], image_number)

In [ ]:
image = images[image_number]

In [ ]:
horizontal_flip = cv2.flip(image, 1)
vertical_flip = cv2.flip(image, 0)

In [ ]:
display_image(horizontal_flip, image_number)
display_image(vertical_flip, image_number)

In [ ]:
rotation_angle = 270
rows, cols = image.shape
rotation_matrix = cv2.getRotationMatrix2D((cols/2, rows/2), rotation_angle, 1)
rotated_image = cv2.warpAffine(image, rotation_matrix, (cols, rows))

display_image(rotated_image, image_number)

In [ ]:
alpha = 1.8
beta = 0
contrast_image = cv2.convertScaleAbs(image, alpha=alpha, beta=beta)

display_image(contrast_image, image_number)

In [ ]:
brightness_factor = 120
brightness_image = cv2.add(image, np.full(image.shape, brightness_factor, dtype=np.uint8))

display_image(brightness_image, image_number)